In [1]:
from langchain_community.utilities import SQLDatabase

# Path to your SQLite database
db_path ="tpch_testdb-main/db/tpch120k_sqlite3.db"

# Connect to the database
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

# Print database details
print("Database Dialect:", db.dialect)

# Get all usable tables
print("Tables in the Database:", db.get_usable_table_names())

# Run a test query 
query = "SELECT * FROM customer LIMIT 10;"  
print("Query Results:", db.run(query))

Database Dialect: sqlite
Tables in the Database: ['customer', 'lineitem', 'nation', 'orders', 'part', 'partsupp', 'region', 'supplier']
Query Results: [(1, 'Customer#000000001', 'j5BkijBM1PkCy0O1m', 15, '25-989-741-2988', 711.56, 'BUILDING', 'AN0PL zMkz5kz6N64kM wnAxz6MMgx5B6Nl1ljnM4i QmA4ixB nMgB11jQRi61z70 76NlmCMNPg1yhQjR7myP0'), (2, 'Customer#000000002', '75S7P5k5ClzmQ', 13, '23-768-687-3665', 121.65, 'AUTOMOBILE', '7O2hQQAmzC0mzMxQCnAxNkAPQ40Ly R'), (3, 'Customer#000000003', '407LO15gnn6Q45MNymCwwLx1OC73Qz', 1, '11-719-748-3364', 7498.12, 'AUTOMOBILE', 'y0gRP7h OwL3M6CCSxySRk4hym nP5lhR'), (4, 'Customer#000000004', 'yzwwR1BS537kRzN0nQ4hkx4', 4, '14-128-190-5944', 2866.83, 'MACHINERY', '75MgBC06n3gk4zlwnLgN24yB61SgOA2QwgM 2xlnnPlAxNQ25l14x213wnAkBB64C'), (5, 'Customer#000000005', 'm4nnNlwB35BN4y263gBz0mQh74m50h', 3, '13-750-942-6364', 794.47, 'HOUSEHOLD', 'ROh40lCjSPyLRSnzyjjjB5mwn3S4xklC4LiQlzh RjCLNOBBgl74L2wj1mN3LPzli3iPNjBSg0R707CSn5QAlLSyAnx6nw4nNQw OA2z5333'), (6, 'Customer#0

In [18]:


api_key = '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'


from langchain.chat_models import init_chat_model
llm = init_chat_model("meta-llama/llama-4-maverick-17b-128e-instruct", model_provider="groq", api_key=api_key, verbose=False)
response=llm.invoke("What is sql language?")
print(response.content.strip())

**SQL (Structured Query Language)**

SQL is a standard programming language designed for managing and manipulating data stored in relational database management systems (RDBMS). It is used to perform various operations, such as creating and modifying database structures, inserting, updating, and deleting data, and querying data to retrieve specific information.

**Key Features of SQL:**

1. **Data Definition Language (DDL)**: SQL provides commands to define and modify database structures, such as creating and dropping tables, indexes, and views.
2. **Data Manipulation Language (DML)**: SQL provides commands to insert, update, and delete data in a database.
3. **Data Query Language (DQL)**: SQL provides commands to retrieve data from a database, such as SELECT statements.
4. **Data Control Language (DCL)**: SQL provides commands to control access to a database, such as GRANT and REVOKE statements.

**Common SQL Operations:**

1. **CREATE**: Create a new table or database object.
2. **IN

In [19]:
from typing_extensions import TypedDict


class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str
    

In [20]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)


In [21]:
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.prompts import PromptTemplate
# 1. Define CLEAN few-shot examples (question → query only)
few_shot_examples = [
    {"input": "List all artists.", "query": "SELECT * FROM Artist;"},
    {
        "input": "Find all albums for the artist 'AC/DC'.",
        "query": "SELECT * FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'AC/DC');",
    }
        
]

# 2. Create example prompt template
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "Question: {input}"),
    ("ai", "SQLQuery: {query}")  # Only show question → SQL mapping
])

# 3. Create few-shot prompt template
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=few_shot_examples,
)

# 4. Define FINAL prompt template (single definition)
query_prompt_template = ChatPromptTemplate.from_messages([
    (
        "system",
        """You are a SQL expert. Generate SQL queries following these rules:
        1. Use {dialect} syntax
        2. Only use these tables:
        {table_info}
        3. No explanations
        4. Limit to {top_k} results"""
    ),
    few_shot_prompt,
    ("human", "Question: {input}"),  
])

# 5. Format with ALL required variables
formatted_prompt = query_prompt_template.format(
    dialect=db,
    table_info=db.get_usable_table_names(),
    top_k=10,
    input="question?"  
)

print(formatted_prompt)

System: You are a SQL expert. Generate SQL queries following these rules:
        1. Use <langchain_community.utilities.sql_database.SQLDatabase object at 0x000001F3AECC5990> syntax
        2. Only use these tables:
        ['customer', 'lineitem', 'nation', 'orders', 'part', 'partsupp', 'region', 'supplier']
        3. No explanations
        4. Limit to 10 results
Human: Question: List all artists.
AI: SQLQuery: SELECT * FROM Artist;
Human: Question: Find all albums for the artist 'AC/DC'.
AI: SQLQuery: SELECT * FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'AC/DC');
Human: Question: question?


In [77]:
from typing_extensions import Annotated



class QueryOutput(TypedDict):
    """Generated SQL query."""
    query: Annotated[str, ..., "Syntactically valid SQL query."]
    

def write_query(state: State):
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"],
            "few_shot_examples": formatted_prompt,
            
        })
    structured_llm = llm.with_structured_output(QueryOutput)
    
    

    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}

In [78]:
input="what order cost more than $3000?"

In [81]:
query=write_query(State(question=input))
print(query)

{'query': 'SELECT * FROM orders WHERE O_TOTALPRICE > 3000 LIMIT 10'}


In [82]:
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool


def execute_query(state: State):
    """Execute SQL query in a tabular format."""
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    return {"result": execute_query_tool.invoke(state["query"])}

In [83]:
execute_query({"query": query})

{'result': "[(1, 4, 'O', 125405.67, '1996-01-02', '5-LOW', 'Clerk#000000951', 0, 'jSmgxOi157kxm gCwQzgNOmiO0MkhCR4N BLj6OQBkl34kj2yly5RzlOzx0A1Chi2j'), (2, 8, 'O', 23768.89, '1996-12-01', '1-URGENT', 'Clerk#000000880', 0, 'O31m7Sw6LmC25k3RBw BmRkkjmzBCO4Qij67CN3mQ4g5jLhSAlh1CAgR'), (3, 13, 'F', 171938.11, '1993-10-14', '5-LOW', 'Clerk#000000955', 0, 'xxSzlw2hwnM10PijiOz'), (4, 14, 'O', 3924.8, '1995-10-11', '5-LOW', 'Clerk#000000124', 0, 'h3OQgm02ykNOwPxhl xRMjOhSh7j7NMLQxO QiAOCRw3l1n0x5mgyO 5i4mNmO1 mQMwB4SmQmO3n'), (5, 5, 'F', 102684.74, '1994-07-30', '5-LOW', 'Clerk#000000925', 0, 'kP35gMk63NQO60AlClPOim MQzSgLwN3hzL'), (7, 14, 'O', 135813.47, '1996-01-10', '2-HIGH', 'Clerk#000000470', 0, 'LxAgMC N7OkSAmgj2zxhNCAgRNyO7RNC'), (32, 7, 'O', 137139.85, '1995-07-16', '2-HIGH', 'Clerk#000000616', 0, 'MAB50lSS5gniPR34mM16PABjh43ykOSL07zQ01iRlmSwyBz1C  m'), (33, 7, 'F', 55513.67, '1993-10-27', '3-MEDIUM', 'Clerk#000000409', 0, 'Azig7CjSAC7AlLQxj0zlLBShM6hLS M2'), (34, 13, 'O', 83360.53, '1

In [84]:
from os import getenv


key='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'
llm2=ChatOpenAI(model="nvidia/llama-3.3-nemotron-super-49b-v1:free",openai_api_key=key,openai_api_base="https://openrouter.ai/api/v1",temperature=0, verbose=False)


In [94]:
def generate_answer(state: State):
    """Generate a final answer based on the SQL result."""
    prompt = (
        "You are a helpful assistant. Based on the SQL result, answer the user's question.\n"
        "DO NOT include any <think> sections, explanations, or reasoning.\n"
        "ONLY return the final answer in one plain English sentence.\n"
        f'Question: {state["question"]}\n'
        f'SQL Query: {state["query"]}\n'
        f'SQL Result: {state["result"]}'

    )
    responce = llm2.invoke(prompt)
    return {"answer":responce.content}

In [95]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence(
    [write_query, execute_query, generate_answer]
)
graph_builder.add_edge(START, "write_query")
graph = graph_builder.compile()

In [97]:
for step in graph.stream(
    {"question":input}, stream_mode="updates"
):
    print(step)

{'write_query': {'query': 'SELECT * FROM orders WHERE O_TOTALPRICE > 3000 LIMIT 10'}}
{'execute_query': {'result': "[(1, 4, 'O', 125405.67, '1996-01-02', '5-LOW', 'Clerk#000000951', 0, 'jSmgxOi157kxm gCwQzgNOmiO0MkhCR4N BLj6OQBkl34kj2yly5RzlOzx0A1Chi2j'), (2, 8, 'O', 23768.89, '1996-12-01', '1-URGENT', 'Clerk#000000880', 0, 'O31m7Sw6LmC25k3RBw BmRkkjmzBCO4Qij67CN3mQ4g5jLhSAlh1CAgR'), (3, 13, 'F', 171938.11, '1993-10-14', '5-LOW', 'Clerk#000000955', 0, 'xxSzlw2hwnM10PijiOz'), (4, 14, 'O', 3924.8, '1995-10-11', '5-LOW', 'Clerk#000000124', 0, 'h3OQgm02ykNOwPxhl xRMjOhSh7j7NMLQxO QiAOCRw3l1n0x5mgyO 5i4mNmO1 mQMwB4SmQmO3n'), (5, 5, 'F', 102684.74, '1994-07-30', '5-LOW', 'Clerk#000000925', 0, 'kP35gMk63NQO60AlClPOim MQzSgLwN3hzL'), (7, 14, 'O', 135813.47, '1996-01-10', '2-HIGH', 'Clerk#000000470', 0, 'LxAgMC N7OkSAmgj2zxhNCAgRNyO7RNC'), (32, 7, 'O', 137139.85, '1995-07-16', '2-HIGH', 'Clerk#000000616', 0, 'MAB50lSS5gniPR34mM16PABjh43ykOSL07zQ01iRlmSwyBz1C  m'), (33, 7, 'F', 55513.67, '1993-1